Integrantes:

- André Dylan Andrade (11832426)
- João Guilherme J. Marinho (10698193)
- Luiz Fernando Rabelo (11796893)
- Marcos Antonio Nobre Coutinho (10716397)

# Introdução

Esse notebook é um projeto de aprendizado de máquina elaborado para o *Trabalho 03* da disciplina *SCC0630 - Inteligência Artificial*. O objetivo é aplicar algoritmos tradicionais de classificação em uma base de dados.

O [banco de dados "Bank Customer Chunk"](https://www.kaggle.com/datasets/radheshyamkollipara/bank-customer-churn?select=Customer-Churn-Records.csv) foi encontrado no Kaggle.

Entender os motivos de **churn** (abandono de algum produto) é muito importante para muitas empresas, a fim de conhecer públicos-alvo e otimizar campanhas de retenção, por exemplo. No banco de dados selecionado, as informações são, supostamente, de usuários de banco (instituição financeira).

# Pré-processamento e visualização dos dados

Antes de aplicar os algoritmos é necessário explorar um pouco o dataset.

Faremos algumas visualizações dos dados para entender o comportamento das variáveis condicionadas as classes (**CHURN** e **~CHURN**) de interesse. Também serão realizadas algumas transformações com o objetivo de ajustar os dados para entrada dos algoritmos.

Abaixo importamos as bibliotecas e funções utilizadas durante o projeto.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.preprocessing import MinMaxScaler

from sklearn import tree
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

Importamos os dados.

In [ ]:
df = pd.read_csv('Customer-Churn-Records.csv')
df.head(5)

Iniciamos fazendo uma descrição das variáveis originais:

- **RowNumber**: número da linha (index);
- **CustomerID**: identificador do banco;
- **Surname**: sobrenome do usuário;
- **CreditScore**: score do usuário;
- **Geography**: localização (país) do usuário;
- **Gender**: gênero do usuário;
- **Age**: idade do usuário;
- **Tenure**: número de anos que o usuário é cliente do banco;
- **Balance**: balanço no cartão de crédito do usuário;
- **NumOfProducts**: número de produtos do usuário;
- **HasCrCard**: se o usuário tem cartão de crédito do banco;
- **IsActiveMember**: se é um usuário ativo do banco;
- **EstimatedSalary**: salário estimado do usuário;
- **Exited**: se o usuário saiu do banco **(churn)** ou não;
- **Complain**: se houve reclamação do usuário ou não;
- **Satisfaction Score**: satisfação do usuário para resolução de problemas;
- **Card Type**: tipo de cartão de crédito do usuário;
- **Points Earned**: pontos ganhos do usuário por usar o cartão de crédito;












A coluna **RowNumber** é apenas uma duplicação dos índices, e a coluna **CostumerID** representa apenas um idenficador aleatório. Ambas colunas não ajudam a explicar o **churn** e serão excluídas. Também renomearemos as variáveis **Satisfaction Score**, **Card Type** e **Points Earned** retirando seus espaços, a fim de facilitar a utilização de alguns métodos abaixo.

In [ ]:
df.drop(columns=['RowNumber','CustomerId'], inplace=True)
df.rename(columns={'Satisfaction Score': 'SatisfactionScore'}, inplace=True)
df.rename(columns={'Card Type': 'CardType'}, inplace=True)
df.rename(columns={'Point Earned': 'PointEarned'}, inplace=True)
df.head(5)

Agora fazemos uma sumarização das variáveis para começar a entende-las.

Verificamos o número de linhas e colunas, além do tipo das variáveis.

In [ ]:
print('Número de linhas e colunas do dataframe: ', df.shape, '\n')
print(df.dtypes)

Existem algumas variávies categóricas (*object*) que posteriormente serão mapeadas como variáveis numéricas, uma alteração necessária para a aplicação de alguns algoritmos.

Verificamos se há valores faltantes ou duplicados.

In [ ]:
print(df.isna().sum())

In [ ]:
len(df[df.duplicated()])

Não há valores faltantes os duplicados.

# Visualização dos dados

### Exited (churn)

Essa é a variável que temos interesse em fazer a predição.

Vamos alterar o tipo da variável para evitar a confusão em relação ao significado dos valores $0$ e $1$, facilitando a interpretação dos resultados abaixo. Posteriormente reverteremos o mapeamento para aplicação dos algoritmos.

In [ ]:
df['Exited'] = df['Exited'].map({0: '~CHURN', 1: 'CHURN'})

In [ ]:
exited = df.Exited.value_counts() / sum(df.Exited.value_counts())
exited

Observamos que aproximadamente $20\%$ dos usuários saíram do banco.

### Surname

Segundo os dados informados no Kaggle, a variável **Surname** não tem impacto em relação ao usuário deixar ou não o banco. Vamos explorar a variável para decidir se será ou não excluída.

In [ ]:
surname = df.Surname.value_counts()
sns.histplot(surname, binwidth=1, binrange=[min(surname), max(surname)])
plt.ylabel('Nº de sobrenomes')
plt.xlabel('Usuários por sobrenome')
plt.show()

Observa-se que a maior parte dos sobrenomes possui poucos usuários no dataset, ou seja, há uma dificuldade de generalização com tão poucas informações. Dessa forma não vamos levar em consideração a variável para análise.

In [ ]:
df.drop(columns=['Surname'], inplace=True)
df.head(5)

### Variáveis quantitativas

Abaixo plotamos os boxplots de algumas variáveis contínuas condicionados pela variável **Exited**, a fim de termos uma noção se é possível explicar a mesma por alguma das outras variáveis.

In [ ]:
variables = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'PointEarned', 'Exited']

plt.figure(figsize=(13.5,10))
for i in range(len(variables) - 1):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='Exited', y=variables[i], data=df)

Percebemos que as variáveis, de modo geral, apresentam pouca diferença entre os grupos que saíram (ou não) do banco. A maior diferença é observada na variável **Age**, em que observamos que o **churn** acontece com usuários mais velhos. Observamos que a variável **Tenure** apresenta maior variabilidade entre os usuários que saíram do banco. Além disso, vemos que a variável **Balance** apresenta valores consideravelmente maiores para usuários que saíram do banco.

In [ ]:
nop1 = df[df['NumOfProducts'] == 1]
nop2 = df[df['NumOfProducts'] == 2]
nop3 = df[df['NumOfProducts'] == 3]
nop4 = df[df['NumOfProducts'] == 4]

In [ ]:
nop1.Exited.value_counts() / sum(nop1.Exited.value_counts())

In [ ]:
nop2.Exited.value_counts() / sum(nop2.Exited.value_counts())

In [ ]:
nop3.Exited.value_counts() / sum(nop3.Exited.value_counts())

In [ ]:
nop4.Exited.value_counts() / sum(nop4.Exited.value_counts())

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2, 2, 1)
nop1.Exited.value_counts().plot(kind='bar', title='Churn Clientes com 1 Produto', rot=0);
plt.subplot(2, 2, 2)
nop2.Exited.value_counts().plot(kind='bar', title='Churn Clientes 2 Produtos', rot=0);
plt.subplot(2, 2, 3)
nop3.Exited.value_counts().plot(kind='bar', title='Churn Clientes com 3 Produtos', rot=0);
plt.subplot(2, 2, 4)
nop4.Exited.value_counts().plot(kind='bar', title='Churn Clientes com 4 Produtos', rot=0);

Há grandes diferenças no **churn** de acordo com o número de produtos. Todos usuários com $4$ produtos saíram do banco. Mais de $80\%$ usuários com $3$ produtos saíram do banco. Usuários com menos produtos saem proporcionalmente menos do banco.

### Variáveis qualitativas

#### País

In [ ]:
france = df[df['Geography'] == 'France']
germany = df[df['Geography'] == 'Germany']
spain = df[df['Geography'] == 'Spain']

In [ ]:
france.Exited.value_counts() / sum(france.Exited.value_counts())

In [ ]:
germany.Exited.value_counts() / sum(germany.Exited.value_counts())

In [ ]:
spain.Exited.value_counts() / sum(spain.Exited.value_counts())

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1, 3, 1)
france.Exited.value_counts().plot(kind='bar', title='Churn Clientes na França', rot=0);
plt.subplot(1, 3, 2)
germany.Exited.value_counts().plot(kind='bar', title='Churn Clientes na Alemanha', rot=0);
plt.subplot(1, 3, 3)
spain.Exited.value_counts().plot(kind='bar', title='Churn Clientes na Espanha', rot=0);

Observamos resultado parecido para usuários da França e Espanha, enquanto usuários da Alemanha tem maior tendência de deixar o banco em relação aos outros países.

Vamos fazer um mapeamento da variável para uma variável categórica, uma necessidade para aplicação de alguns algoritmos.

In [ ]:
df['Geography'] = df['Geography'].map({'France': 1, 'Germany': 2, 'Spain': 3})

#### Gênero

In [ ]:
female = df[df['Gender'] == 'Female']
male = df[df['Gender'] == 'Male']

In [ ]:
female.Exited.value_counts() / sum(female.Exited.value_counts())

In [ ]:
male.Exited.value_counts() / sum(male.Exited.value_counts())

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
female.Exited.value_counts().plot(kind='bar', title='Churn Clientes do Gênero F', rot=0);
plt.subplot(1, 2, 2)
male.Exited.value_counts().plot(kind='bar', title='Churn Clientes do Gênero M', rot=0);

Observamos que proporcionalmente há maior saída do banco entre mulheres do que entre homens.

Vamos fazer um mapeamento da variável para uma variável categórica, uma necessidade para aplicação de alguns algoritmos.

In [ ]:
df['Gender'] = df['Gender'].map({'Female': 1, 'Male': 2})

#### Cartão de Crédito

In [ ]:
noCrCard = df[df['HasCrCard'] == 0]
CrCard = df[df['HasCrCard'] == 1]

In [ ]:
noCrCard.Exited.value_counts() / sum(noCrCard.Exited.value_counts())

In [ ]:
CrCard.Exited.value_counts() / sum(CrCard.Exited.value_counts())

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
noCrCard.Exited.value_counts().plot(kind='bar', title='Churn Clientes sem Cartão', rot=0);
plt.subplot(1, 2, 2)
CrCard.Exited.value_counts().plot(kind='bar', title='Churn Clientes com Cartão', rot=0);

Observa-se proporção parecida de **churn** entre ambos os grupos.

#### Atividade

In [ ]:
noActiveMember = df[df['IsActiveMember'] == 0]
ActiveMember = df[df['IsActiveMember'] == 1]

In [ ]:
noActiveMember.Exited.value_counts() / sum(noActiveMember.Exited.value_counts())

In [ ]:
ActiveMember.Exited.value_counts() / sum(ActiveMember.Exited.value_counts())

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
noActiveMember.Exited.value_counts().plot(kind='bar', title='Churn Clientes Inativos', rot=0);
plt.subplot(1, 2, 2)
ActiveMember.Exited.value_counts().plot(kind='bar', title='Churn Clientes Ativos', rot=0);

Observa-se maior **churn** entre os usuários que não estão ativos.

#### Reclamação

In [ ]:
noComplain = df[df['Complain'] == 0]
Complain = df[df['Complain'] == 1]

In [ ]:
noComplain.Exited.value_counts() / sum(noComplain.Exited.value_counts())

In [ ]:
Complain.Exited.value_counts() / sum(Complain.Exited.value_counts())

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
noComplain.Exited.value_counts().plot(kind='bar', title='Churn Clientes sem Reclamações', rot=0);
plt.subplot(1, 2, 2)
Complain.Exited.value_counts().plot(kind='bar', title='Churn Clientes com Reclamações', rot=0);

Observa-se que a variável *Complain* (o usuário ter feito ou não reclamação) é o que mais diferencia os usuários. Quase todos usuários que fizeram reclamações saíram do banco, enquanto poucos usuários saem do banco sem ter feito uma reclamação antes.

#### Tipo do Cartão

In [ ]:
silver = df[df['CardType'] == 'SILVER']
gold = df[df['CardType'] == 'GOLD']
diamond = df[df['CardType'] == 'DIAMOND']
platinum = df[df['CardType'] == 'PLATINUM']

In [ ]:
silver.Exited.value_counts() / sum(silver.Exited.value_counts())

In [ ]:
gold.Exited.value_counts() / sum(gold.Exited.value_counts())

In [ ]:
platinum.Exited.value_counts() / sum(platinum.Exited.value_counts())

In [ ]:
diamond.Exited.value_counts() / sum(diamond.Exited.value_counts())

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2, 2, 1)
silver.Exited.value_counts().plot(kind='bar', title='Churn Clientes com Cartão Silver', rot=0);
plt.subplot(2, 2, 2)
gold.Exited.value_counts().plot(kind='bar', title='Churn Clientes com Cartão Gold', rot=0);
plt.subplot(2, 2, 3)
platinum.Exited.value_counts().plot(kind='bar', title='Churn Clientes com Cartão Platinum', rot=0);
plt.subplot(2, 2, 4)
diamond.Exited.value_counts().plot(kind='bar', title='Churn Clientes com Cartão Diamond', rot=0);

Não se observa grandes diferenças de **churn** entre os tipos de cartão.

Vamos fazer um mapeamento da variável para uma variável categórica, uma necessidade para aplicação de alguns algoritmos.

In [ ]:
df['CardType'] = df['CardType'].map({'SILVER': 1, 'GOLD': 2, 'PLATINUM': 3, 'DIAMOND': 4})

#### Nível de Satisfação

In [ ]:
ss1 = df[df['SatisfactionScore'] == 1]
ss2 = df[df['SatisfactionScore'] == 2]
ss3 = df[df['SatisfactionScore'] == 3]
ss4 = df[df['SatisfactionScore'] == 4]
ss5 = df[df['SatisfactionScore'] == 5]

In [ ]:
ss1.Exited.value_counts() / sum(ss1.Exited.value_counts())

In [ ]:
ss2.Exited.value_counts() / sum(ss2.Exited.value_counts())

In [ ]:
ss3.Exited.value_counts() / sum(ss3.Exited.value_counts())

In [ ]:
ss4.Exited.value_counts() / sum(ss4.Exited.value_counts())

In [ ]:
ss5.Exited.value_counts() / sum(ss5.Exited.value_counts())

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(2, 3, 1)
ss1.Exited.value_counts().plot(kind='bar', title='Churn Clientes Satisfação 1', rot=0);
plt.subplot(2, 3, 2)
ss2.Exited.value_counts().plot(kind='bar', title='Churn Clientes Satisfação 2', rot=0);
plt.subplot(2, 3, 3)
ss3.Exited.value_counts().plot(kind='bar', title='Churn Clientes Satisfação 3', rot=0);
plt.subplot(2, 3, 4)
ss4.Exited.value_counts().plot(kind='bar', title='Churn Clientes Satisfação 4', rot=0);
plt.subplot(2, 3, 5)
ss5.Exited.value_counts().plot(kind='bar', title='Churn Clientes Satisfação 5', rot=0);

Não há diferença substancial de **churn** entre as notas de satisfação.

### One-hot-encoding e normalização

O one-hot-encoding transforma uma variável categórica com mais de duas classes em várias variáveis binárias. A quantidade de variáveis criadas é o número de classes da variável original.

Quando fazemos o mapeamento de uma variável categórica (qualitativa) como variável numérica (quantitativa) uma ordem é implicitamente adicionada na variável. Assim sendo, o one-hot-encoding é interessante para as variáveis nominais porque mantém toda informação das classes originais, mas elimina a ordem implícita criada ao mapear um variável categórica como numérica.

In [ ]:
df = pd.get_dummies(df, columns=['Geography'])

Variáveis com ordem de grandeza e/ou variâncias diferentes podem gerar distorções na aplicação dos algoritmos. A normalização busca corrigir essas distorções.

In [ ]:
scaler = MinMaxScaler()
df[['CreditScore','Age','Tenure','Balance','EstimatedSalary','PointEarned','NumOfProducts','SatisfactionScore','CardType']] = scaler.fit_transform(df[['CreditScore','Age','Tenure','Balance','EstimatedSalary','PointEarned','NumOfProducts','SatisfactionScore','CardType']])

In [ ]:
df.head(5)

### Correlação entre variáveis

Revertemos o mapeamento da variável **Exited** para cálculo da correlação e aplicação dos algoritmos.

In [ ]:
df['Exited'] = df['Exited'].map({'~CHURN': 0, 'CHURN': 1})

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(df.corr(), cmap='vlag');
plt.show()

Podemos perceber a fortíssima correlação entre as variáveis **Complain** e **Exited**.

# Aplicação dos algoritmos

In [ ]:
nk = 10 # número de pastas

In [ ]:
Y = df[['Exited']].values.ravel()

df.drop(columns=['Exited'], inplace=True)

X = df.values

## K-Nearest Neighbors (KNN)

O K-Nearest Neighbors (KNN) é um algoritmo de aprendizado de máquina supervisionado usado para problemas de classificação e regressão. A ideia principal do KNN é que objetos semelhantes tendem a estar próximos uns dos outros no espaço de características. O algoritmo faz a predição de uma nova observação com base nos rótulos das observações de treinamento mais próximas a ele.

É importante testar diferentes valores de *k*, a fim de evitar casos de underfitting e overfitting. Também é interessante treinar o modelo com diferentes métricas de distância. Para cada problema, uma métrica diferente pode  tornar o algoritmo mais eficiente.




Aplicamos o algoritmo KNN com quatro diferentes métricas de distância (*euclidiana*, *manhattan*, *chebyshev* e *minhowski*). Mais valores de $k$ foram testados do que os valores apresentados abaixo, entretanto, para as quatro métricas a medida F1-Score continuava decrescendo para valores maiores de $k$. Então optamos por deixar na versão final valores de $k$ de $1$ a $10$.


In [ ]:
def KNN(X, Y, nk, k_range):
    metrics = ['euclidean', 'manhattan', 'chebyshev', 'minkowski']

    for metric in metrics:
        vk = []
        vmetric = []
        vscore = []
        for k in range(1, k_range, 2):
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
            cv = cross_validate(knn, X, Y, scoring = 'f1', cv=nk)
            vscore.append(cv['test_score'].mean())
            vk.append(k)
            vmetric.append(metric)

        plt.figure(figsize=(6,4))
        plt.plot(vk, vscore, '-bo')
        plt.xlabel('k')
        plt.ylabel('F1 Score')
        plt.title(metric)
        plt.show(True)
        best_k = vk[np.argmax(vscore)]
        acc = vscore[np.argmax(vscore)]
        print('Melhor k:', best_k)
        print('F1 Score:', acc)

In [ ]:
KNN(X, Y, nk, 11)

Observamos que a métrica *manhattan* foi a com pior desempenho, mas ainda assim bom com F1-Score = $0.9951$. Para as outras métricas, obtivemos F1-Score = $0.9966$. Esse valor também foi encontrado em outros algoritmos abaixo, bem como foi o melhor valor para o dataset. Para as métricas *chebyshev* e *minkowski*, esse valor foi encontrado com $k = 3$, enquanto para distância euclidiana o valor de $k$ foi $5$.

## Naive-Bayes

O algoritmo Naive Bayes é um método de classificação probabilístico baseado no Teorema de Bayes, que descreve a probabilidade condicional de um evento dado o conhecimento prévio sobre as condições relacionadas a ele. O Naive Bayes assume que as características dos dados são independentes entre si, o que é uma simplificação forte (por isso, Naive - ingênuo), mas útil em muitos casos. O algoritmo Naive Bayes é rápido e eficiente, mesmo em grandes conjuntos de dados. Ele é especialmente adequado para problemas de classificação com múltiplas classes e alta dimensionalidade. Além disso, o Naive Bayes é menos suscetível a overfitting em comparação com outros algoritmos mais complexos.

Para o algoritmo de Naive-Bayes não precisamos selecionar hiperparâmetros. O que devemos fazer é escolher algumas variações dos modelos que supõem diferentes distribuições condicionais para as variáveis.


Consideramos as distribuições Normal (Gaussiana), Bernoulli, Multinomial e *Complement* (uma variação do Multinomial).




In [ ]:
def NaiveBayes(X, Y, nk):
    model = GaussianNB()
    cv = cross_validate(model, X, Y, scoring = 'f1', cv = nk)
    print(f"Gaussian: F1-Score = {cv['test_score'].mean()}")

    model = BernoulliNB()
    cv = cross_validate(model, X, Y, scoring = 'f1', cv = nk)
    print(f"Bernoulli: F1-Score = {cv['test_score'].mean()}")

    model = MultinomialNB()
    cv = cross_validate(model, X, Y, scoring = 'f1', cv = nk)
    print(f"Multinimial: F1-Score = {cv['test_score'].mean()}")

    model = ComplementNB()
    cv = cross_validate(model, X, Y, scoring = 'f1', cv = nk)
    print(f"Complement: F1-Score = {cv['test_score'].mean()}")

In [ ]:
NaiveBayes(X, Y, nk)

Percebemos que o modelo Complement foi o que apresentou pior resultado, sendo também inferior aos modelos KNN vistos acima. As demais distribuições proporcionaram resultados iguais aos melhores do KNN.



## Árvore de Decisão

Uma árvore de decisão é uma estrutura hierárquica composta por nós e arestas. Cada nó representa uma decisão ou um teste a ser realizado sobre um atributo específico, enquanto as arestas representam o resultado desse teste. Convencionalmente, arestas à esquerda são direcionadas a resultados verdadeiros e arestas à direita a resultados falsos. O nó raiz da árvore representa o atributo mais importante para fazer a primeira decisão, enquanto os nós folha representam as classes ou valores previstos.

Existem vários critérios de divisão em árvores de decisão, os quais podem afetar no processo de construção da árvore. Nesse sentido, faremos testes considerando diferentes critérios: Impureza de Gini (mede a probabilidade de classificar erroneamente uma amostra aleatória, ou seja, quanto menor a impureza de Gini, melhor a qualidade da divisão - mais "pura" ela é) e Entropia (mede a quantidade média de informação necessária para descrever a classificação de uma amostra, ou seja, quanto maior a entropia, mais impura é a divisão).

Além dos critérios de divisão, também podemos variar o número de estimadores considerados para cada critério. Apesar de, teoricamente, se aumentarmos o número de estimadores geralmente melhoramos o desempenho preditivo e a capacidade de generalização do modelo, percebemos um ponto de saturação além do qual o aumento no número de estimadores não forneceu benefícios significativos na classificação. Assim, consideraremos apenas quantidades de estimadores no intervalo [5,25].

In [ ]:
def tree_decision(X, Y, nk, max_depth=None, n_estimator_a=2, n_estimator_z=12, n_estimator_r=3):
    # Define as métricas utilizadas e números de estimadores
    metrics = ['gini', 'entropy']
    estimators = [i for i in range(n_estimator_a, n_estimator_z, n_estimator_r)]

    scores = {'bagging': (0, '', 0), 'ada': (0, '', 0), 'random': (0, '', 0)}

    # Para cada métrica
    for metric in metrics:
        # Chama o classificador para diferentes métricas
        dtc = tree.DecisionTreeClassifier(criterion=metric, max_depth=max_depth, random_state=101)
        # Para cada valor de estimador:
        for n_estimators in estimators:
            # Bagging Classifier
            model_bagging = BaggingClassifier(estimator=dtc, n_estimators=n_estimators)
            cv_bagging = cross_validate(model_bagging, X, Y, scoring='f1', cv=nk)
            scores['bagging'] = max(scores['bagging'], (cv_bagging['test_score'].mean(), metric, n_estimators))
            # Ada Boost Classifier
            model_ada_boost = AdaBoostClassifier(estimator=dtc, n_estimators=n_estimators, learning_rate=1)
            cv_ada_boost = cross_validate(model_ada_boost, X, Y, scoring='f1', cv=nk)
            scores['ada'] = max(scores['ada'], (cv_ada_boost['test_score'].mean(), metric, n_estimators))
            # Random Forest Classifier
            model_random_forest = RandomForestClassifier(n_estimators = n_estimators)
            cv_random_forest = cross_validate(model_random_forest, X, Y, scoring='f1', cv=nk)
            scores['random'] = max(scores['random'], (cv_random_forest['test_score'].mean(), metric, n_estimators))
        plt.figure(figsize=(15,6))
        plt.title(f'{metric}')
        tree.plot_tree(dtc.fit(X,Y), filled=True, class_names=['~Churn', 'Churn'], feature_names=df.columns)
        plt.show(True)

    print(scores)
    print("Legenda: {'classificador': (score, critério, número_estimadores)}")

In [ ]:
tree_decision(X, Y, nk)

Podemos "podar" a árvore de decisão, definindo uma profundidade máxima, a fim de simplificar a visualização do resultado. Como o atributo do nó raiz e os atributos de seus primeiros descendentes de menor grau segmentam bem os clientes em relação ao Churn, não ovtivemos prejuízos significativos na pontuação.

In [ ]:
tree_decision(X, Y, nk, max_depth=3)

Percebemos que os classificadores Random Forest e Bagging obtiveram um melhor desempenho, seguidos do Ada Boost. Observamos que na saída acima que os melhores resultados foram encontrados utilizando a medida *gini*. Os valores da medida F1-Score encontrada para os classificadores Bagging e Random Forest foram os mesmos dos melhores resultados dos classificadores Naive-Bayes. Foram testados valores maiores para o número de estimadores, mas observamos que poucos estimadores já produzem o melhor resultado.

## Multilayer Perceptron

O Multilayer Perceptron (MLP) é um tipo de rede neural artificial, um modelo de aprendizado de máquina inspirado no funcionamento do cérebro humano. É composto por várias camadas de neurônios artificiais, incluindo uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída.

A camada de entrada seriam os dados que fornecemos. A camada de saída são as predições do algoritmo. As camadas ocultas realizam transformações nos dados que produzirão a camada de saída.

Podemos selecionar a função de ativação utilizada no algoritmo. Cada neurônio aplica uma função de ativação à soma ponderada das suas entradas. Podem ser utilizadas funções não-lineares a fim de que o medelo aprenda relações complexas nos dados.

Dentro do modelo também existem os pesos e viés (bias) associados a cada neurõnio. Na função utilizada não precisamos informar os pesos e viés, sendo necessário apenas definir o parâmetro *solver* que seleciona um algoritmo que busca otimizar a escolha dos pesos e viés.

Ajustaremos o modelo usando diferentes *solvers* (*lbfgs*,*sgd*,*adam*) e *funções de ativação* (*identidade*, *logística*, *tangente hiperbólica* e *ReLU*).

Foram testados diferentes quantidades de camadas e neurônios por camadas alterando o parâmetro **hidden_layer_sizes**. Mais camadas e neurônios por camadas não ajudaram no aumento do poder de predição. Assim sendo, deixamos os resultados obtidos com apenas uma camada e um neurônio.

In [ ]:
def MultilayerPerceptron(X,Y,nk,max_iter = 1000,hidden_layer_sizes=(1)):
    solvers = ['lbfgs','sgd','adam']
    activations = ['identity', 'logistic', 'tanh', 'relu']
    for solver in solvers:
        for activation in activations:
            model = MLPClassifier(solver=solver, alpha=1e-5, activation=activation, max_iter = max_iter,
                            hidden_layer_sizes = hidden_layer_sizes, random_state=1)
            cv = cross_validate(model, X, Y, scoring = "f1", cv = nk)
            print(f"{activation}--{solver}: F1-score = {cv['test_score'].mean()}")

In [ ]:
MultilayerPerceptron(X, Y, nk)

Podemos observar que os melhores resultados foram encontrados para todas as permutações de *funções de ativação* com os *solvers* *lbfgs* e *adam*. Para o *solver* *sgd* o resultado foi péssimo.

# Seleção de variáveis e reaplicação dos algoritmos

O teste de permutação é um teste estatístico não exato sobre se as médias de duas variáveis são iguais. No nosso problema, a variável 1 é uma variável de interesse X condicionada ao usuário ter saído do banco, e a variável 2 é a mesma variável X condicionada ao usuário não ter saído do banco. Se as variáveis não apresentam mesma média (verificado com valor-p "baixo"), então a variável X pode nos ajudar com a predição.

In [ ]:
def permutation_test(data,variable):
    num_permutations = 10_000
    v_name = variable

    v1 = np.array(data[data['Exited'] == 1][variable])
    v2 = np.array(data[data['Exited'] == 0][variable])
    observed_statistic = np.mean(v1) - np.mean(v2)

    permutation_stats = np.zeros(num_permutations)
    combined_data = np.concatenate([v1, v2])
    for i in range(num_permutations):
        np.random.shuffle(combined_data)
        permuted_v1 = combined_data[:len(v1)]
        permuted_v2 = combined_data[len(v1):]
        permutation_stats[i] = np.mean(permuted_v1) - np.mean(permuted_v2)

    p_value = np.sum(permutation_stats >= observed_statistic) / num_permutations
    print(f"{v_name}: Valor-p = {p_value}")
    return

O teste Qui-Quadrado testa a hipótese de duas variáveis quantitativas serem independentes. No nosso problema, vamos estar comparando uma variável X de interesse com a variável *Exited*. A não independência (verificada com valor-p "baixo") nos indica que a variável X pode nos ajudar com a predição.

In [ ]:
def chi_squared_test(data,v1):
    contingency_table = pd.crosstab(data[v1], Y)
    _, p, _, _ = chi2_contingency(contingency_table)
    print(f"{v1}: Valor-p = {p}")
    return

Novamente, lemos o dataset.

In [ ]:
df_reduced = pd.read_csv('Customer-Churn-Records.csv')
df_reduced.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

Vamos considerar o nível de significância $\alpha = 0.05$, ou seja, variáveis com $valor-p \leq 0.05$ serão utilizadas nos algoritmos.

In [ ]:
variables = ['CreditScore', 'Age', 'Tenure','Balance', 'EstimatedSalary', 'Point Earned','NumOfProducts']
for variable in variables:
    permutation_test(df_reduced,variable)

Utilizeramos as variáveis **Age** e **Balance**.

In [ ]:
df_reduced.drop(columns=['CreditScore', 'Tenure', 'EstimatedSalary', 'Point Earned', 'NumOfProducts'], inplace=True)

In [ ]:
variables = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'Complain', 'Satisfaction Score', 'Card Type']
for variable in variables:
    chi_squared_test(df_reduced,variable)

Utilizaramos as variáveis **Geography**, **Gender**, **IsActiveMember** e **Complain**.

In [ ]:
df_reduced.drop(columns=['HasCrCard', 'Satisfaction Score', 'Card Type'], inplace=True)

## Reaplicação dos algoritmos

In [ ]:
df_reduced.columns

Novamente mapeamos as variáveis necessárias.

In [ ]:
df_reduced['Geography'] = df_reduced['Geography'].map({'France': 1, 'Germany': 2, 'Spain': 3})

In [ ]:
df_reduced['Gender'] = df_reduced['Gender'].map({'Female': 1, 'Male': 2})

Novamente fazemos one-hot-encoding e normalização.

In [ ]:
df_reduced = pd.get_dummies(df_reduced, columns=['Geography'])

In [ ]:
scaler = MinMaxScaler()
df_reduced[['Age','Balance']] = scaler.fit_transform(df_reduced[['Age', 'Balance']])

Dividimos os dados novamente.

In [ ]:
df_reduced.columns

In [ ]:
X = df_reduced[['Gender', 'Age', 'Balance', 'IsActiveMember', 'Complain', 'Geography_1', 'Geography_2', 'Geography_3']].values
Y = df_reduced[['Exited']].values.ravel()

Aplicamos os algoritmos novamente.

In [ ]:
KNN(X, Y, nk, 11)

In [ ]:
NaiveBayes(X, Y, nk)

In [ ]:
tree_decision(X, Y, nk, max_depth=3)

In [ ]:
MultilayerPerceptron(X, Y, nk)

De forma geral podemos dizer que foi uma boa alternativa a seleção de variáveis realizada. Alguns algoritmos perderam poder de predição, enquanto outros melhoraram o poder de predição, mas nada substancial. O ponto positivo é que ao reduzir variáveis reduzimos a complexidade do modelo e entendemos melhor quais variáveis estavam ou não nos ajudando na predição. Além disso, mesmo com a seleção das variáveis ainda há muitos algoritmos atingindo o maior valor de F1-Score encontrado anteriormente ($0.9966$).

## Considerações Finais

Ao longo deste projeto, foram explorados a aplicação de diferentes algoritmos de aprendizado de máquina para a predição da saída de clientes de um banco. Os algoritmos KNN, Naive Bayes, Árvores de Decisão e Multilayer Perceptron foram aplicados para realizar a predição. No entanto, ao avaliar os resultados, notamos um empecilho relacionado ao desbalanceamento nos dados, especialmente devido à presença do atributo "complain".

A análise exploratória do dataset revelou que quase todos os clientes que apresentaram reclamações acabaram deixando o banco. Esse padrão levou a pontuações de precisão próximas a 100% em todos os algoritmos testados. Entretanto, a alta performance observada pode ser atribuída, em grande parte, ao desequilíbrio na distribuição dos dados.

Considerando a possibilidade de viés nos resultados, seria recomendado conduzir uma análise sem a inclusão da variável "complain", uma vez que o modelo pode ter aprendido a associar a presença de reclamações a uma alta probabilidade de churn, ignorando outros fatores relevantes.
